# `nanoGPT`: Shakespeare

## Install / Setup

### Google Colab

```python
from google.colab import drive
drive.mount('/content/drive')
```

### First Time Running

We need to install `wordplay` and setup the Shakespeare dataset

This will need to be ran the first time you are running this notebook.

Following the

```python
!python3 -m pip install wordplay
```

you will need to restart your runtime (Runtime -> Restart runtime)

After this, you should be able to

```python
>>> import ngpt
>>> ngpt.__file__
'/content/nanoGPT/src/ngpt/__init__.py'
```

In [1]:
%%bash

python3 -c 'import wordplay; print(wordplay.__file__)' 2> '/dev/null'

if [[ $? -eq 0 ]]; then
    echo "Has wordplay installed. Nothing to do."
else
    echo "Does not have wordplay installed. Installing..."
    git clone 'https://github.com/saforem2/wordplay'
    python3 wordplay/data/shakespeare_char/prepare.py
    python3 wordplay/data/shakespeare/prepare.py
    python3 -m pip install deepspeed
    python3 -m pip install -e wordplay
fi

/content/wordplay/src/wordplay/__init__.py
Has wordplay installed. Nothing to do.


## Post Install

If installed correctly, you should be able to:

```python
>>> import wordplay
>>> wordplay.__file__
'/path/to/wordplay/src/wordplay/__init__.py'
```

In [2]:
import os
# os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
os.environ['COLORTERM'] = 'truecolor'

In [3]:
%load_ext autoreload
%autoreload 2

import wordplay
from enrich import get_logger
log = get_logger(level='INFO')
#from rich import print
log.info(wordplay.__file__)

INFO:root:/content/wordplay/src/wordplay/__init__.py


[2024-02-06 14:13:55][INFO][<ipython-input-3-df3b64ce7190>:8] - /content/wordplay/src/wordplay/__init__.py


## Build Trainer

Explicitly, we:

1. `setup_torch(...)`
2. Build `cfg: DictConfig = get_config(...)`
3. Instnatiate `config: ExperimentConfig = instantiate(cfg)`
4. Build `trainer = Trainer(config)`

In [4]:
import os
import numpy as np
from ezpz import setup
from hydra.utils import instantiate
from wordplay.configs import get_config, PROJECT_ROOT
from wordplay.trainer import Trainer

HF_DATASETS_CACHE = PROJECT_ROOT.joinpath('.cache', 'huggingface')
HF_DATASETS_CACHE.mkdir(exist_ok=True, parents=True)

os.environ['HF_DATASETS_CACHE'] = HF_DATASETS_CACHE.as_posix()

BACKEND = 'DDP'

rank = setup(
    framework='pytorch',
    backend=BACKEND,
    seed=1234,
)

cfg = get_config(
    [
        'data=shakespeare',
        'model=shakespeare',
        'optimizer=shakespeare',
        'train=shakespeare',
        f'train.backend={BACKEND}',
        'train.compile=false',
        'train.dtype=float16',
        'train.max_iters=5000',
        'train.log_interval=100',
        'train.eval_interval=500',
    ]
)
config = instantiate(cfg)

[2024-02-06 14:13:58][INFO][configs:72] - Setting HF_DATASETS_CACHE to /content/wordplay/.cache/huggingface/datasets

[2024-02-06 14:13:59][INFO][utils:148] - Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.

[2024-02-06 14:13:59][INFO][utils:160] - NumExpr defaulting to 8 threads.

[2024-02-06 14:14:00][INFO][dist:257] - DistInfo={
    "DEVICE": "cuda",
    "DEVICE_ID": "cuda:0",
    "DISTRIBUTED_BACKEND": "nccl",
    "GPUS_PER_NODE": 1,
    "HOSTFILE": "/content/hostfile",
    "HOSTNAME": "83594087f236",
    "HOSTS": "['83594087f236']",
    "LOCAL_RANK": 0,
    "MACHINE": "83594087f236",
    "NGPUS": 1,
    "NODE_ID": 0,
    "NUM_NODES": 1,
    "RANK": 0,
    "SCHEDULER": "LOCAL",
    "WORLD_SIZE_IN_USE": 1,
    "WORLD_SIZE_TOTAL": 1
}

[2024-02-06 14:14:00][INFO][dist:642] - [0/1] Using device='cuda' with backend='DDP' + 'nccl' for distributed training.

...

[2024-02-06 14:14:01][INFO][configs:308] - Loading train from /content/wordplay/data/shakespeare_char/train.bin

[2024-02-06 14:14:01][INFO][configs:283] - Rescaling GAS -> GAS // WORLD_SIZE = 1 // 1

[2024-02-06 14:14:01][INFO][configs:433] - Tokens per iteration: 16,384

[2024-02-06 14:14:01][INFO][configs:456] - Using self.ptdtype=torch.float16 on self.device_type='cuda'

[2024-02-06 14:14:01][INFO][configs:462] - Initializing a new model from scratch

In [5]:
trainer = Trainer(config)

[2024-02-06 14:14:01][INFO][trainer:277] - Initializing a new model from scratch

[2024-02-06 14:14:01][INFO][model:255] - number of parameters: 10.65M

[2024-02-06 14:14:01][INFO][model:445] - num decayed parameter tensors: 26, with 10,740,096 parameters

[2024-02-06 14:14:01][INFO][model:449] - num non-decayed parameter tensors: 13, with 4,992 parameters

[2024-02-06 14:14:02][INFO][model:465] - using fused AdamW: True

[2024-02-06 14:14:02][CRITICAL][trainer:338] - "devid='cuda:0'"

## Prompt (**prior** to training)

In [6]:
query = "What is an LLM?"
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=128,
    top_k=1,
    display=False
)
log.info(f"['prompt']: '{query}'")
log.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[2024-02-06 14:14:12][INFO][<ipython-input-6-c240c2a7e644>:9] - ['prompt']: 'What is an LLM?'

[2024-02-06 14:14:12][INFO][<ipython-input-6-c240c2a7e644>:10] - ['response']:

What is an LLM??GGGGGVVVVVVVVVVVYYYYYYYYYCCCCCCCCCCCCCCCCCCCCCCCCCCCCCiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii

## Train Model

|  name  |       description            |
|:------:|:----------------------------:|
| `step` | Current training step        |
| `loss` | Loss value                   |
| `dt`   | Time per step (in **ms**)    |
| `sps`  | Samples per second           |
| `mtps` | (million) Tokens per sec     |
| `mfu`  | Model Flops utilization[^1]  |
^legend: #tbl-legend

[^1]: in units of A100 `bfloat16` peak FLOPS

In [7]:
trainer.config.device_type

'cuda'

In [8]:
trainer.train()

  0%|          | 0/5000 [00:00<?, ?it/s]

                              Training Legend                               
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃    abbr    ┃ desc                                                        ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    step    │ Current training iteration                                  │
│    loss    │ Loss value                                                  │
│     dt     │ Elapsed time per training step (measured in **ms**)         │
│    dtf     │ Elapsed time per forward step (measured in **ms**)          │
│    dtb     │ Elapsed time per backward step (measured in **ms**)         │
│    sps     │ Samples per second                                          │
│    mtps    │ Tokens per second, measured in MEGA (1 x 10^6) tokens / sec │
│    mfu     │ Model flops utilization                                     │
│ train_loss │ Training loss value                                         │
│  val_loss  │ Validation loss value                                       │
└────────────┴─────────────────────────────────────────────────────────────┘

  2%|▏         | 97/5000 [00:07<02:07, 38.41it/s]

[2024-02-06 14:14:30][INFO][trainer:768] - step=100 loss=2.4807 dt=28.3995 dtf=8.2712 dtb=16.9648 sps=35.2119 mtps=0.5769 mfu=13.1208 train_loss=4.3000 val_loss=4.2911

  4%|▍         | 197/5000 [00:10<02:07, 37.72it/s]

...

 98%|█████████▊| 4897/5000 [02:57<00:02, 37.93it/s]

[2024-02-06 14:17:20][INFO][trainer:768] - step=4900 loss=0.8081 dt=25.8349 dtf=8.3901 dtb=14.3042 sps=38.7073 mtps=0.6342 mfu=14.5014 train_loss=0.6439 val_loss=1.6704

100%|█████████▉| 4997/5000 [03:00<00:00, 38.24it/s]

[2024-02-06 14:17:23][INFO][trainer:768] - step=5000 loss=0.8423 dt=26.0989 dtf=8.5888 dtb=14.1429 sps=38.3158 mtps=0.6278 mfu=14.4790 train_loss=0.6439 val_loss=1.6704

100%|██████████| 5000/5000 [03:00<00:00, 27.72it/s]


## Evaluate Model

In [9]:
import time

query = "What is an LLM?"
t0 = time.perf_counter()
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=2,
    display=False
)
log.info(f'took: {time.perf_counter() - t0:.4f}s')
log.info(f"['prompt']: '{query}'")
log.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[2024-02-06 14:25:22][INFO][<ipython-input-9-99b80f46fd99>:12] - took: 1.8314s

[2024-02-06 14:25:22][INFO][<ipython-input-9-99b80f46fd99>:13] - ['prompt']: 'What is an LLM?'

[2024-02-06 14:25:22][INFO][<ipython-input-9-99b80f46fd99>:14] - ['response']:

What is an LLM?

POLIXENES:
That when my father's grace is mine.

PAULINA:
The matter, then, like a child, and a forty of
The queen's life is a man. Hermione, she came from
the morning: he is a man that made the law, and that
he was like a man dishessed to his charters;